# Append AOU
- Include AOU and solubility to the O2 dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import gsw
from multiprocessing import pool

In [8]:
version='1.1.1.1.1.1'

In [9]:
diro2='/glade/campaign/univ/ugit0034/ML4O2_results/'
dirTS='/glade/campaign/univ/ugit0034/EN4/L09_20x180x360/'

In [10]:
def calc_o2sat(year,mon):
    mo=str(mon).zfill(2)
    fn=f'{dirTS}/EN4_TSN2_L09_180x360_{year}{mo}.nc'
    ds=xr.open_dataset(fn)
    t=ds.CT#.to_numpy()
    s=ds.SA#.to_numpy()
    p=ds.depth#.to_numpy()
    lon=ds.lon#.to_numpy()
    lat=ds.lat#.to_numpy()
    o2sat=gsw.O2sol(s,t,p,lon,lat)
    return o2sat

In [12]:
def calc_aou(ver):
    fn=f'{diro2}O2map_v{ver}.nc'
    wn=f'{diro2}O2AOUmap_v{ver}.nc'
    ds=xr.open_dataset(fn)
    time = ds.time.to_numpy()
    Nt=np.size(time)
    aou=np.zeros((Nt,20,180,360))
    for n,tim in enumerate(time):
        year=str(np.datetime64(tim,'Y'))
        mon=n%12+1
        o2sat = calc_o2sat(year,mon)
        aou[n,:,:,:] = o2sat.to_numpy()-ds.o2est.isel(time=n).to_numpy()
    #
    p=ds.depth#.to_numpy()
    lon=ds.lon#.to_numpy()
    lat=ds.lat#.to_numpy()
    da=xr.DataArray(data=aou,name='aouest',
                    dims=['time','depth','lat','lon'],
                    coords={'time':time,'depth':p,'lat':lat,'lon':lon})
    ds['aouest']=da
    ds.to_netcdf(wn)

In [13]:
calc_aou(version)